# Hyperparameter Tuning 과제

- Manual Search
- Random Search (Grid Search 참고)
- Bayesian Optimization using HyperOpt
<br>연습해보기!

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.datasets import load_iris
import warnings
warnings.filterwarnings('ignore')
import time
from functools import partial
from sklearn.model_selection import cross_val_score

In [4]:
iris = load_iris()

X = iris.data
y = iris.target

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=23)

## 1. Manual Search
사이킷런 Logistic Regresstion https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html?highlight=logistic%20regression#sklearn.linear_model.LogisticRegression

In [11]:
'''
maunal 1 
'''

manual1 = LogisticRegression(
    # 첨부한 링크에 있는 사이킷런 도큐먼트를 참고하여 원하는대로 하이퍼파라미터 값을 넣어주세요
    C = 1,
    max_iter = 1000,
    random_state = 101
)

manual1.fit(X_train,y_train)
y_pred = manual1.predict(X_train)

# 정확도를 계산해주세요
accuracy_score(y_train, y_pred)

0.9642857142857143

In [12]:
'''
maunal 2 
'''
# 다양한 하이퍼파라미터 값을 사용하여 또 다른 manual search를 해보세요
# manual1 에 비해 정확도가 더 높아지면 더 좋습니다 !

manual2 = LogisticRegression(
    # 첨부한 링크에 있는 사이킷런 도큐먼트를 참고하여 원하는대로 하이퍼파라미터 값을 넣어주세요
    C = 5,
    max_iter = 3000,
    solver = 'lbfgs'
)

manual2.fit(X_train,y_train)
y_pred = manual2.predict(X_train)

# 정확도를 계산해주세요
accuracy_score(y_train, y_pred)


0.9821428571428571

## 2. Grid Search

In [21]:
from sklearn.model_selection import GridSearchCV

# 파라미터 후보 설정
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100],
              'max_iter' : [100, 500, 1000, 1500, 2000]}

# 그리드 서치 호출
grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5, return_train_score=True)   

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=23)

start = time.time()
# 모델 적합
grid_search.fit(X_train, y_train)

# 모델 평가
print(f"test set score : {grid_search.score(X_test, y_test)}")

print(f"best parameters: {grid_search.best_params_}")
print(f"best CV score: {grid_search.best_score_}")

# 소요 시간 측정
print("time :", time.time() - start)

test set score : 1.0
best parameters: {'C': 1, 'max_iter': 100}
best CV score: 0.9553359683794467
time : 4.53711724281311


## 3. Random Search
grid search 코드와 매우 유사하니 참고해서 빈칸 채워주세요 !

In [22]:
from sklearn.model_selection import RandomizedSearchCV

# 파라미터 후보 설정
param_rand = {'C':[0.1,0.5,1,5,10,20],
              'max_iter':[100,500,1000,3000,5000,10000],
              'solver':['saga','lbfgs','saga','lbfgs','saga','lbfgs']}
# 그리드 서치 호출
randomized_search = RandomizedSearchCV(LogisticRegression(), param_rand, cv=6, return_train_score =True )
# 데이터 분할
X_trn,X_tst,y_trn,y_tst = train_test_split(X,y,random_state=25)
# 모델 적합
randomized_search.fit(X_trn,y_trn)
starttime=time.time()
# 모델 평가
print(f"test set score : {randomized_search.score(X_tst,y_tst)}")
print(f"best parameters: {randomized_search.best_params_}")
print(f"best CV score : {randomized_search.best_score_}")
# 소요 시간 측정

print("time:",time.time()-starttime)

test set score : 0.9210526315789473
best parameters: {'solver': 'saga', 'max_iter': 10000, 'C': 10}
best CV score : 0.9912280701754387
time: 0.0019407272338867188


## 4. HyperOpt

In [2]:
!pip install hyperopt

In [15]:
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials

space = {
    'penalty' : hp.choice('penalty', ['l1', 'l2', 'elasticnet', 'none']), 
    'fit_intercept' : hp.choice('fit_intercept', [True, False]),
    'max_iter' : hp.uniform('max_iter', 100,1000)
}

def objective(space):
    model = LogisticRegression(# 채워주세요
      C = 5,
      max_iter = 3000,
      solver = 'saga'                         
                              )
    
    accuracy = cross_val_score(model, X_train, y_train, cv = 4).mean() # 채워주세요

    return {'loss': -accuracy, 'status': STATUS_OK }# 채워주세요
    
trials = Trials()
best = fmin(fn= objective,
            space= space,
            algo= tpe.suggest,
            max_evals = 20,
            trials= trials)
best

100%|██████████| 20/20 [00:02<00:00,  9.39it/s, best loss: -0.9732142857142858]


{'fit_intercept': 1, 'max_iter': 444.34962682167395, 'penalty': 2}

In [17]:
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import accuracy_score

pen = {0: 'l1', 1: 'l2', 2: 'elasticnet', 3: None}
incpt = {0: True, 1: False}

trained = LogisticRegression(# 채워주세요
      C = 5,
      max_iter = 3000,
      solver = 'saga' ``
                            ).fit(X_train,y_train)

pred = trained.predict(X_test)
accuracy_score(y_test, pred)

# 정확도 도출 부분까지 코드 짜시면 됩니다!

0.9736842105263158

## 선택) Bayesian Optimization 사용


In [ ]:
!pip install bayesian-optimization

In [ ]:
from bayes_opt import BayesianOptimization